Test bodo snowflake haversine

In [1]:
%autopx

%autopx enabled


In [1]:
import snowflake.connector
#import bodo
import time
from sqlalchemy.dialects import registry
registry.register('snowflake', 'snowflake.sqlalchemy', 'dialect')

In [2]:
import os
with open("./snowflake-secrets.txt","r") as f:
    for line in f:
        args=line.split("=")
        os.environ[args[0]]=args[1].strip()
username=os.environ["username"]
password=os.environ["password"]
account=os.environ["account"]
warehouse="BODO_VW"
database="BODO_DB"
role="BODO_R1"

In [3]:
import bodo
import pandas as pd
@bodo.jit()
def run_test4():
    df=pd.read_sql('select * from PUBLIC.TRIPS limit 10000', f"snowflake://{username}:{password}@{account}/{database}/public?role={role}&warehouse={warehouse}",)
    print(len(df))
    return df
df=run_test4()

10000


In [4]:
import numpy as np
@bodo.jit()
def haversine_np(x):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [x["start_station_longitude"], x["start_station_latitude"], x["end_station_longitude"], x["end_station_latitude"]])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

In [5]:
@bodo.jit()
def calculuate_haversine(df):
    df["distance_in_km"]=df.apply(haversine_np,axis=1)
    return df
df=calculuate_haversine(df)

In [7]:

@bodo.jit()
def calc_test4(df):
    df["startday"]=pd.to_datetime(df['starttime']).dt.date
    gdf=df.groupby(["startday","usertype"],as_index=False).agg({"tripduration":["mean","median"],"distance_in_km":["sum"]})
    return gdf
gdf=calc_test4(df)
gdf=gdf.droplevel(level=0,axis=1)

In [8]:
gdf

,,,mean,median,sum
0,2015-03-29,Subscriber,1208.141414,486.0,139.972567
1,2015-03-29,Customer,2945.416667,3054.0,40.026099
2,2018-05-31,Subscriber,766.214876,610.5,3314.061354
3,2018-05-31,Customer,1335.132653,1234.5,188.264213
4,2018-03-06,Subscriber,641.785603,470.0,873.852004
5,2018-03-06,Customer,1297.893617,1339.0,85.131012
6,2016-10-01,Subscriber,820.413333,624.0,539.838795
7,2016-10-01,Customer,1541.968254,1269.0,95.916544
8,2015-12-22,Subscriber,760.768421,562.0,655.550140
9,2015-12-22,Customer,2365.695652,1150.0,29.769882
